In [1]:
# Imports using Sklearn make shortcut functions
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, OneHotEncoder
from sklearn.pipeline import make_pipeline
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from google.cloud import bigquery
from datetime import datetime, timedelta

In [2]:
# Print current time
now = datetime.now()
current_time = now.strftime("%Y-%m-%d %H:%M:%S")

In [3]:
# Imports data
data = pd.read_csv('../raw_data/1988_to_2024_combined_clean_polling_and_results.csv')

In [4]:
# Renames randmom PERCENTAGE COLUMN - FIX IN DATA CLEANING
data.rename(columns={'OTH_PERCENTAGE': 'OTH_ACT'}, inplace=True)

In [5]:
# Convert date columns to datetime
data['enddate'] = pd.to_datetime(data['enddate'])
data['next_elec_date'] = pd.to_datetime(data['next_elec_date'])
data['startdate'] = pd.to_datetime(data['startdate'])

In [6]:
# Handle numerical transformer
num_columns_selector = ['samplesize', 'months_to_elec_weight']
num_transformer = MinMaxScaler()

In [7]:
# Handle categorical transformer
cat_columns_selector = ['rating']
cat_transformer = make_pipeline(OrdinalEncoder(categories = [['F','F+','E-','E','E+','D-','D','D+','C-','C','C+','B-','B','B+','A-']]),MinMaxScaler())

In [8]:
# One-hot encoder for party_in_power
ohe = OneHotEncoder()

In [9]:
# Build the preprocessing pipeline
preproc_pipeline = make_column_transformer(
    (num_transformer, num_columns_selector),
    (cat_transformer, cat_columns_selector),
    (ohe,['party_in_power']),
    remainder='passthrough',
    verbose_feature_names_out=False
)

In [10]:
data['next_elec_date'].value_counts()

next_elec_date
2015-05-07    1930
2024-07-04    1362
2010-05-06     584
1992-04-09     453
2019-12-12     440
1997-05-01     338
2017-06-08     254
2001-06-07     213
2005-05-05      98
Name: count, dtype: int64

In [11]:
# Define election dates
election_date_2019 = datetime.strptime('2019-12-12', '%Y-%m-%d')
cutoff_date = election_date_2019 - timedelta(days=84)
prediction_date = election_date_2019 - timedelta(days=24)

In [12]:
# Split data
data_train = data[data['startdate'] > '2003-12-31']
data_train = data_train[data_train['startdate'] < cutoff_date]
data_test_1 = data[(data['startdate'] >= cutoff_date) & (data['startdate'] < prediction_date)]
data_test = data_test_1[data_test_1['next_elec_date'] == election_date_2019]


In [13]:
# Fit transform preprocessing pipeline to data_train
data_train_processed = preproc_pipeline.fit_transform(data_train)


In [14]:
# Transform preprocessing pipeline to data_test
data_test_processed = preproc_pipeline.transform(data_test)

In [15]:
# Check feature names make sense for data_train
data_train_processed = pd.DataFrame(
    data_train_processed, columns=preproc_pipeline.get_feature_names_out()
)

# Check feature names make sense for data_test
data_test_processed = pd.DataFrame(
    data_test_processed, columns=preproc_pipeline.get_feature_names_out()
)

In [16]:
data_train_processed.columns

Index(['samplesize', 'months_to_elec_weight', 'rating',
       'party_in_power_Conservative', 'party_in_power_Labour', 'Unnamed: 0',
       'startdate', 'enddate', 'pollster', 'next_elec_date', 'days_to_elec',
       'months_to_elec', 'poll_length', 'CON_FC', 'LAB_FC', 'LIB_FC', 'BRX_FC',
       'GRE_FC', 'OTH_FC', 'PLC_FC', 'SNP_FC', 'UKI_FC', 'CON_ACT', 'LAB_ACT',
       'LIB_ACT', 'BRX_ACT', 'GRE_ACT', 'PLC_ACT', 'SNP_ACT', 'UKI_ACT',
       'OTH_ACT'],
      dtype='object')

In [17]:
# Define our X by dropping irrelevant and y columns

X_train = data_train_processed.drop(columns=['startdate', 'enddate', 'pollster', 'Unnamed: 0', 'next_elec_date', 'days_to_elec', 'months_to_elec', 'party_in_power_Labour', 'LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT'])
X_test = data_test_processed.drop(columns=['startdate', 'enddate', 'pollster', 'Unnamed: 0', 'next_elec_date', 'days_to_elec', 'months_to_elec', 'party_in_power_Labour', 'LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT'])

In [18]:
# Build our target matrix
y_train = data_train_processed[['next_elec_date', 'LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT']]
y_test = data_test_processed[['next_elec_date', 'LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT']]

In [19]:
# Drops y_train actuals where the actual is the actual result we are trying to predict, replaces with NaNs
y_train.loc[y_train['next_elec_date'] == '2019-12-12',
         ['LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT']] = np.nan

In [20]:
X_test.describe()

,samplesize,months_to_elec_weight,rating,party_in_power_Conservative,poll_length,CON_FC,LAB_FC,LIB_FC,BRX_FC,GRE_FC,OTH_FC,PLC_FC,SNP_FC,UKI_FC
count,60.000000,60.000000,60.0,60.0,60,60.00,60.00,60.00,56.00,60.00,60.0,60.00,60.00,39.0
unique,51.000000,3.000000,4.0,1.0,8,16.00,13.00,12.00,13.00,9.00,5.0,2.00,4.00,3.0
top,0.037283,0.831387,1.0,1.0,1,0.37,0.29,0.15,0.12,0.03,0.0,0.01,0.04,0.0
freq,3.000000,30.000000,37.0,60.0,27,9.00,12.00,11.00,11.00,23.00,25.0,36.00,35.00,24.0


In [21]:
# Calculates average median value of X_test
averages = X_test.mean()
averages

samplesize                     0.039575
months_to_elec_weight          0.776239
rating                         0.904762
party_in_power_Conservative         1.0
poll_length                         2.2
CON_FC                         0.379167
LAB_FC                           0.2665
LIB_FC                         0.168333
BRX_FC                         0.092321
GRE_FC                         0.041833
OTH_FC                         0.009333
PLC_FC                            0.006
SNP_FC                           0.0375
UKI_FC                         0.004615
dtype: object

In [22]:
# Creates imputation values for y_train to impute over actuals we are trying to predict
imputation_values = {
    'CON_ACT': averages['CON_FC'],
    'LAB_ACT': averages['LAB_FC'],
    'LIB_ACT': averages['LIB_FC'],
    'BRX_ACT': averages['BRX_FC'],
    'GRE_ACT': averages['GRE_FC'],
    'OTH_ACT': averages['OTH_FC'],
    'PLC_ACT': averages['PLC_FC'],
    'SNP_ACT': averages['SNP_FC'],
    'UKI_ACT': averages['UKI_FC']
}

In [23]:
# Applies imputation values to y_train
y_train = y_train.fillna(value=imputation_values)


/var/folders/kb/jn3km5pj6lnd7dfkgljcjy300000gn/T/ipykernel_29291/3014033841.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_train = y_train.fillna(value=imputation_values)


In [24]:
# Instantiate the model for each party
models = {}
parties = ['CON', 'LAB', 'LIB', 'BRX', 'GRE', 'SNP', 'UKI', 'PLC', 'OTH']
for party in parties:
    models[party] = XGBRegressor(
        learning_rate=0.3, n_estimators=300, max_depth=3, subsample=0.7,
        objective='reg:squarederror', nthread=-1, enable_categorical=True
    )

In [25]:
# Train models
X_train_matrix = np.array(X_train)
for party in parties:
    models[party].fit(X_train_matrix, y_train[f'{party}_ACT'])


In [26]:
# Evaluate feature importance for each party model
# for i, x in enumerate(X_train.columns):
#     print(i, ' : ', x)
# for party in parties:
#     plt.bar(range(len(models[party].feature_importances_)), models[party].feature_importances_)
#     plt.xlabel(party)
#     plt.show()

In [27]:
# # Evaluate model performance with cross validation
# cv_scores = {}
# for party in parties:
#     cv_scores[party] = cross_val_score(models[party], X_train_matrix, y_train[f'{party}_ACT']).mean()

In [28]:
actuals_2019 = {
    "CON": 0.4378,  # Conservative Party
    "LAB": 0.3267,  # Labour Party
    "LIB": 0.1164,  # Liberal Democrats
    "BRX": 0.0200,  # Brexit Party
    "GRE": 0.0254,  # Green Party
    "SNP": 0.0379,  # Scottish National Party
    "UKI": 0.0000,  # UK Independence Party
    "PLC": 0.0051,  # Plaid Cymru
    "OTH": 0.0307   # Other parties
}

In [29]:
# Evaluate predictions
X_test_matrix = np.array(X_test)
mean_predictions = {}
for party in parties:
    mean_predictions[party] = models[party].predict(X_test_matrix).mean()


In [30]:
averages

samplesize                     0.039575
months_to_elec_weight          0.776239
rating                         0.904762
party_in_power_Conservative         1.0
poll_length                         2.2
CON_FC                         0.379167
LAB_FC                           0.2665
LIB_FC                         0.168333
BRX_FC                         0.092321
GRE_FC                         0.041833
OTH_FC                         0.009333
PLC_FC                            0.006
SNP_FC                           0.0375
UKI_FC                         0.004615
dtype: object

In [31]:
# Print mean predictions
for party in parties:
    print(f"2019 mean prediction for {party}: {mean_predictions[party]}")
    print(f"2019 actual result for {party}  : {actuals_2019[party]}")

2019 mean prediction for CON: 0.4005642235279083
2019 actual result for CON  : 0.4378
2019 mean prediction for LAB: 0.29026132822036743
2019 actual result for LAB  : 0.3267
2019 mean prediction for LIB: 0.1591961830854416
2019 actual result for LIB  : 0.1164
2019 mean prediction for BRX: 0.0629538893699646
2019 actual result for BRX  : 0.02
2019 mean prediction for GRE: 0.03725437819957733
2019 actual result for GRE  : 0.0254
2019 mean prediction for SNP: 0.0359082855284214
2019 actual result for SNP  : 0.0379
2019 mean prediction for UKI: 0.010248980484902859
2019 actual result for UKI  : 0.0
2019 mean prediction for PLC: 0.005723768379539251
2019 actual result for PLC  : 0.0051
2019 mean prediction for OTH: 0.013436107896268368
2019 actual result for OTH  : 0.0307


In [32]:
averages['CON_FC']

0.37916666666666676

In [33]:
# Evaluate last five polls
last_predictions = {}
for party in parties:
    last_predictions[party] = models[party].predict(X_test_matrix)[-5:]

In [34]:
# Print mean of predictions
for party in parties:
    print(f"2019 last prediction for {party}: {last_predictions[party].mean()}")
    print(f"2019 actual result for {party}  : {actuals_2019[party]}")


2019 last prediction for CON: 0.39079517126083374
2019 actual result for CON  : 0.4378
2019 last prediction for LAB: 0.30251994729042053
2019 actual result for LAB  : 0.3267
2019 last prediction for LIB: 0.15667665004730225
2019 actual result for LIB  : 0.1164
2019 last prediction for BRX: 0.06764575839042664
2019 actual result for BRX  : 0.02
2019 last prediction for GRE: 0.03048955835402012
2019 actual result for GRE  : 0.0254
2019 last prediction for SNP: 0.03536443039774895
2019 actual result for SNP  : 0.0379
2019 last prediction for UKI: 0.009905876591801643
2019 actual result for UKI  : 0.0
2019 last prediction for PLC: 0.005601013079285622
2019 actual result for PLC  : 0.0051
2019 last prediction for OTH: 0.012515529990196228
2019 actual result for OTH  : 0.0307
